<a id="TableOfContents"></a>
# TABLE OF CONTENTS:
<li><a href='#imports'>Imports</a></li>
<li><a href="#Q1">Question 1</a></li>
<li><a href='#Q2'>Question 2</a></li>
<li><a href='#Q3'>Question 3</a></li>
<li><a href="#Q4">Question 4</a></li>

<a id='imports'></a>
# IMPORTS:
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# Vectorization and tables
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Stats
from scipy import stats

# sklearn
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

# Datasets
from pydataset import data

# .py files
import feature_engineering

<a id='Q1'></a>
# Question 1:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 1. Load the tips dataset.

In [2]:
tips = data('tips')
tips.sample()

,total_bill,tip,sex,smoker,day,time,size
169,10.59,1.61,Female,Yes,Sat,Dinner,2


##### 1a. Create a column named price_per_person. This should be the total bill divided by the party size.

In [3]:
tips['price_per_person'] = round(tips.total_bill / tips['size'], 2)
tips.sample()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
67,16.45,2.47,Female,No,Sat,Dinner,2,8.22


##### 1b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

- Everything seems like it would affect the tip amount...  Judging from most important to least important...
    - size
    - time
    - day
    - sex
    - smoker

##### 1c. Use select k best to select the top 2 features for predicting tip amount. What are they?

In [4]:
tips.time.value_counts()

Dinner    176
Lunch      68
Name: time, dtype: int64

In [5]:
tips.sex = np.where(tips.sex == 'Male', 1, 0)
tips.smoker = np.where(tips.smoker == 'Yes', 1, 0)
tips.day = np.where(tips.day == 'Sat', 7, tips.day)
tips.day = np.where(tips.day == 'Sun', 1, tips.day)
tips.day = np.where(tips.day == 'Thur', 5, tips.day)
tips.day = np.where(tips.day == 'Fri', 6, tips.day)
tips.time = np.where(tips.time == 'Dinner', 0, 1)
tips.sample()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
64,18.29,3.76,1,1,7,0,4,4.57


In [6]:
x_col = tips.drop(columns='total_bill')
y_col = tips.total_bill

In [7]:
kbest = SelectKBest(f_regression, k=2)
_ = kbest.fit(x_col, y_col)
x_col.columns[kbest.get_support()].to_list()

['tip', 'price_per_person']

##### 1d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [8]:
LR = LinearRegression()
rfe = RFE(LR, n_features_to_select=2)
rfe.fit(x_col, y_col)
pd.DataFrame({'rfe_ranking' : rfe.ranking_},
             index=x_col.columns)

,rfe_ranking
tip,2
sex,5
smoker,6
day,4
time,3
size,1
price_per_person,1


IT DOTH BE...
- size
- price_per_person

##### 1e. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?

I daresay...

It would appear as though that as the selection size becomes smaller, then so does the similarities between the two

<a id='Q2'></a>
# Question 2:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [10]:
best = feature_engineering.select_kbest(x_col, y_col, 3)
best

['tip', 'size', 'price_per_person']

<a id='Q3'></a>
# Question 3:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [11]:
top_rfe = feature_engineering.rfe(x_col, y_col, 3)
top_rfe

,rfe_ranking
tip,1
size,1
price_per_person,1
time,2
day,3
sex,4
smoker,5


<a id='Q4'></a>
# Question 4:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [12]:
swiss = data('swiss')
swiss.sample()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Neuveville,76.9,43.5,17,15,5.16,20.6


In [15]:
# Assign predictor and target dfs
predictors = swiss.drop(columns='Fertility')
target = swiss.Fertility

In [17]:
# Run 'select_kbest'
top3kbest = feature_engineering.select_kbest(predictors, target, 3)
top3kbest

['Examination', 'Education', 'Catholic']

In [18]:
# Run 'rfe'
top3rfe = feature_engineering.rfe(predictors, target, 3)
top3rfe

,rfe_ranking
Examination,1
Education,1
Infant.Mortality,1
Agriculture,2
Catholic,3
